In [ ]:
from refactor.AsyncScraper import runFleetScrape
from refactor.TRScraper import scrapeTR
from refactor.dataScraper import runSailorData

from fullfile import adjust_race_id, main, postCalcAdjust
import pandas as pd
from datetime import datetime, timedelta
import mysql.connector
import numpy as np

from openskill.models import PlackettLuce, BradleyTerryFull
from regions import teamRegions

In [2]:

calc_all = True

doScrape = False
doUpload = True

targetSeasons = ['f25']
targetTRSeasons = ['s25']

model = PlackettLuce(beta=25.0/120.0)
targetElo = 1000

if doScrape:
    df_races = runFleetScrape()
else:
    # if running scrapers seperately
    df_races = pd.read_json("racesfr.json")

df_races['raceNum'] = df_races['raceID'].apply(
    lambda id: int(id.split("/")[2][:-1]))  # Numeric part
df_races['raceDiv'] = df_races['raceID'].apply(
    lambda id: id.split("/")[2][-1])  # Division part (e.g., 'A', 'B')
df_races['adjusted_raceID'] = df_races.apply(
    adjust_race_id, axis=1)  # to make combined division combined
df_races['Link'] = df_races['Link'].fillna('Unknown')  # fill empty links
# df_races['key'] = np.where(df_races['Link'] == 'Unknown', df_races['Sailor'], df_races['Link'])
df_races['key'] = df_races.apply(
    lambda row: row['Sailor'] + "-" +
    row['Team'] if row['Link'] == 'Unknown' else row['Link'],
    axis=1
)

df_races['partnerKey'] = df_races.apply(
    lambda row: row['Partner'] + "-" +
    row['Team'] if row['PartnerLink'] == 'Unknown' else row['PartnerLink'],
    axis=1
)

if doScrape:
    df_races_tr, trSailors = scrapeTR()
else:
    # if running scrapers seperately
    df_races_tr = pd.read_json("racesTR.json")

df_races_tr['adjusted_raceID'] = df_races_tr['raceID']
df_races_tr['Scoring'] = 'team'
df_races = df_races.rename(
    {'Date': 'date', 'Regatta': 'regatta'}, axis='columns')
df_races_full = pd.concat([df_races, df_races_tr])

# clean up memory
del df_races, df_races_tr

df_races_full = df_races_full.sort_values(
    ['date', 'raceNum', 'raceDiv']).reset_index(drop=True)

df_sailor_ratings = None
if not calc_all:
    cutoff = (datetime.now() - timedelta(weeks=2))
    df_races_full = df_races_full.loc[df_races_full['date'] > cutoff]
    df_sailor_ratings = pd.read_json("sailors-latest.json")      
    # df_sailor_ratings = pd.read_json("sailors-20250424.json")

if doScrape:
    df_sailor_info = runSailorData()
else:
    df_sailor_info = pd.read_json("sailor_data2.json")


In [3]:
people, df_races_full = main(df_sailor_ratings, df_sailor_info, df_races_full)
people, df_sailors, df_races_full = postCalcAdjust(people, df_races_full)

Currently analyzing race 1000/54992 in s16/ike-geiger-team-race, Date:2016-03-19 00:00:00
Currently analyzing race 2000/54992 in s16/oberg, Date:2016-04-16 00:00:00
Currently analyzing race 3000/54992 in f16/stu-nelson, Date:2016-09-17 00:00:00
Currently analyzing race 4000/54992 in f16/protest, Date:2016-10-09 00:00:00
Currently analyzing race 5000/54992 in s17/barnyard-bizzare, Date:2017-03-04 00:00:00
Currently analyzing race 6000/54992 in s17/admiral-moore-team-race, Date:2017-03-25 00:00:00
Currently analyzing race 7000/54992 in s17/new-england-team-race-fowle, Date:2017-04-08 00:00:00
Currently analyzing race 8000/54992 in s17/sperry-women-west-semis, Date:2017-05-23 00:00:00
Currently analyzing race 9000/54992 in f17/owlapalooza-2017, Date:2017-09-30 00:00:00
Currently analyzing race 10000/54992 in f17/nickerson-2017, Date:2017-10-28 00:00:00
Currently analyzing race 11000/54992 in s18/bob-bavier-team-race, Date:2018-03-03 00:00:00
Currently analyzing race 12000/54992 in s18/sea

/Users/carter/Documents/GitHub/sailing-rating-calculation/fullfile.py:861: RuntimeWarning: Mean of empty slice.
  [r['ratio'] for r in p.races if r['pos'] == 'Skipper' and 'ratio' in r.keys()]).mean())
/Users/carter/.pyenv/versions/3.13.1/lib/python3.13/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/carter/Documents/GitHub/sailing-rating-calculation/fullfile.py:863: RuntimeWarning: Mean of empty slice.
  [r['ratio'] for r in p.races if r['pos'] == 'Crew' and 'ratio' in r.keys()]).mean())


16641


In [42]:
# Create a connection
# connection = mysql.connector.connect(
#     host="localhost",
#     port=3308,
#     user="root",
#     password="password",
#     database="crowsnest",
#     allow_local_infile=True
# )
connection = mysql.connector.connect(
    host="yamabiko.proxy.rlwy.net",
    port=10632,
    user="root",
    password="KloKIjhMAMTlSimKtZTTUeahpdcadPNk",
    database="railway",
    allow_local_infile=True
)
cursor = connection.cursor()

In [ ]:
from fullfile import validPerson
def uploadTeams(df_sailors, df_races_full, people, cursor, connection):

    def getCounts(races):
        # season_counts = defaultdict(int)
        season_counts = {}

        for race in races:
            season = race["raceID"].split("/")[0]
            if season not in season_counts.keys():
                season_counts[season] = {}
            if race['pos'] not in season_counts[season].keys():
                season_counts[season][race['pos']] = 0
            season_counts[season][race['pos']] += 1

        return dict(season_counts)

    # batch = db.batch()
    # col = db.collection('eloTeams')
    teams = []
    predteams = []
    scrape = False
    teamNames = teamRegions.keys()
    lenteams = len(teamNames)

    season_mask = df_sailors['Seasons'].apply(lambda x: not set(x['skipper']).isdisjoint(
        targetSeasons) or not set(x['crew']).isdisjoint(targetSeasons))

    # Explode the Teams column to enable grouping
    df_exploded = df_sailors.loc[season_mask].explode('Teams')

    # Group by team and compute necessary aggregates
    team_stats = df_exploded.groupby('Teams').agg(
        numCurMembers=('Teams', 'count'),
        avgSkipperOrdinal=('srOrd', 'mean'),
        avgCrewOrdinal=('crOrd', 'mean'),
        avgSkipperRatio=('skipperAvgRatio', 'mean'),
        avgCrewRatio=('crewAvgRatio', 'mean')
    )

    # Calculate the average values as in the original code
    team_stats['avg'] = (team_stats['avgSkipperOrdinal'] +
                         team_stats['avgCrewOrdinal']) / 2
    team_stats['avgRatio'] = (
        team_stats['avgSkipperRatio'] + team_stats['avgCrewRatio']) / 2

    team_stats = team_stats.reindex(teamNames, fill_value=0)
    today = datetime.today()

    # team_link_map = df_races_full.dropna().drop_duplicates('Team').set_index('Team')['Teamlink'].to_dict()

    df_cleaned = df_races_full.dropna(
        subset=['Team', 'Teamlink']).drop_duplicates(subset='Team', keep='first')

    # Create a dictionary with 'Team' as the key and 'TeamLink' as the value
    team_link_map = pd.Series(
        df_cleaned.Teamlink.values, index=df_cleaned.Team).to_dict()


    for i, (team, row) in enumerate(team_stats.iterrows()):
        # if team != "MIT":
        #     continue
        print(f"{i}/{len(team_stats)} {team}")
        avg = row['avg']
        avgRatio = row['avgRatio']
        numCurMembers = row['numCurMembers']

        region = teamRegions[team]
        # teamLink = df_races.loc[df_races['Team'] == team, 'Teamlink'].iloc[0]
        # Default to '' if team not found
        teamLink = team_link_map.get(team, '')
        url = f"https://scores.collegesailing.org/schools/{teamLink.split("/")[2]}"

        filtered_people = [p for p in people.values() if team in p.teams]

        members = [{"name": str(p.name),
                    "key": p.key,
                    "gender": p.gender,
                    "year": str(p.year),
                    'teams': list(p.teams),
                    'skipperRating': int(p.sr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                    # 'sr': {'mu': p.sr.mu, 'sigma': p.sr.sigma},
                    'crewRating': int(p.cr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                    'womenSkipperRating': int(p.wsr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                    'womenCrewRating': int(p.wcr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                    'tsr': p.tsr.ordinal(target=targetElo, alpha=200 / model.sigma),
                    'wtsr': p.wtsr.ordinal(target=targetElo, alpha=200 / model.sigma),
                    'tcr': p.tcr.ordinal(target=targetElo, alpha=200 / model.sigma),
                    'wtcr': p.wtcr.ordinal(target=targetElo, alpha=200 / model.sigma),
                    'avgSkipperRatio': float(p.avgSkipperRatio),
                    'avgCrewRatio': float(p.avgCrewRatio),
                    # season -> position -> count (normalized position names)
                    'raceCount': (lambda rc: {s: {pos.title(): cnt for pos, cnt in posd.items()} for s, posd in rc.items()})(getCounts(p.races)),
                    # position -> season -> count (used later when inserting SailorTeams)
                    'raceCounts': (lambda rc_norm, ps: {
                        'skipper': {season: rc_norm.get(season, {}).get('Skipper', 0) for season in list(ps['skipper'])},
                        'crew': {season: rc_norm.get(season, {}).get('Crew', 0) for season in list(ps['crew'])}
                    })( (lambda rc: {s: {pos.title(): cnt for pos, cnt in posd.items()} for s, posd in rc.items()})(getCounts(p.races)), p.seasons ),
                    'seasons': p.seasons,
                    'cross': sum([race.get('cross', 0) for race in p.races]),
                    'outLinks': sum([race['outLinks'] for race in p.races if 'outLinks' in race.keys()]),
                    'skipperRank': int(p.skipperRank),
                    'crewRank': int(p.crewRank),
                    'womenSkipperRank': int(p.womenSkipperRank),
                    'womenCrewRank': int(p.womenCrewRank)
                    } for p in filtered_people]
        
        topRating = 0
        topWomenRating = 0
        topRatingTR = 0
        topWomenRatingTR = 0
        
        topRating = 0
        topSkipperSum = 0
        topCrewsSum = 0
        topWomenRating = 0
        topRatingTR = 0
        topWomenRatingTR = 0

        numTops = 3
        if numCurMembers > 0:
            topSkippers = sorted([p for p in filtered_people
                                  if validPerson(p, 'skipper')
                                  #   if sum([race['cross'] for race in p.races if 'cross' in race.keys()]) > 20
                                  #   and sum([race['outLinks'] for race in p.races if 'outLinks' in race.keys()]) > 70
                                  #   and not set(p.seasons['skipper']).isdisjoint(targetSeasons)
                                  # and sum([p['raceCount'][seas] for seas in targetSeasons if seas in p['raceCount'].keys()]) > 5
                                  ],
                                 key=lambda x: x.sr.ordinal(
                                     target=targetElo, alpha=200 / model.sigma),
                                 reverse=True)
            topSkipperSum = sum([p.sr.ordinal(target=targetElo, alpha=200 / model.sigma)
                                 for p in topSkippers[:numTops]])
            topSkippers = [{'name': p.name, 'key': p.key,
                            'mu': p.sr.mu, 'sigma': p.sr.sigma} for p in topSkippers]

            topCrews = sorted([p for p in filtered_people if validPerson(p, 'crew')
                               #   if sum([race['cross'] for race in p.races if 'cross' in race.keys()]) > 20
                               #   and sum([race['outLinks'] for race in p.races if 'outLinks' in race.keys()]) > 70
                               #   and not set(p.seasons['crew']).isdisjoint(targetSeasons)
                               ],
                              key=lambda x: x.cr.ordinal(
                                  target=targetElo, alpha=200 / model.sigma),
                              reverse=True)
            topCrewsSum = sum([p.cr.ordinal(target=targetElo, alpha=200 / model.sigma)
                               for p in topCrews[:numTops]])
            topCrews = [{'name': p.name, 'key': p.key,
                         'mu': p.cr.mu, 'sigma': p.cr.sigma} for p in topCrews]

            topRating = (topSkipperSum + topCrewsSum) / (numTops * 2)
            # print(topSkippers, topCrews, topRating)

            # Women's
            numTops = 2
            topWomenSkippers = sorted([p['womenSkipperRating'] for p in members
                                       #   if p['skipperRank'] != 0
                                      if p['cross'] > 20
                                      and p['gender'] == 'F'
                                      and not set(p['raceCount'].keys()).isdisjoint(targetSeasons)
                                      and all(['Skipper' in p['raceCount'][seas].keys() for seas in targetSeasons if seas in p['raceCount'].keys()])
                                      and sum([p['raceCount'][seas]['Skipper'] for seas in targetSeasons if seas in p['raceCount'].keys()]) > 5
                                       ], reverse=True)[:numTops]
            topWomenSkipperSum = sum(topWomenSkippers)

            topWomenCrews = sorted([p['womenCrewRating'] for p in members
                                    #    if p['crewRank'] != 0
                                    if p['cross'] > 20
                                    and p['gender'] == 'F'
                                    and not set(p['raceCount'].keys()).isdisjoint(targetSeasons)
                                    and all(['Crew' in p['raceCount'][seas].keys() for seas in targetSeasons if seas in p['raceCount'].keys()])
                                    and sum([p['raceCount'][seas]['Crew'] for seas in targetSeasons if seas in p['raceCount'].keys()]) > 5
                                    ], reverse=True)[:numTops]
            topWomenCrewsSum = sum(topWomenCrews)

            topWomenRating = (topWomenSkipperSum +
                              topWomenCrewsSum) / (numTops * 2)

            # Team racing:

            numTops = 3

            topSkippersTR = sorted([p for p in filtered_people
                                    if p.tsr.mu != model.mu
                                    if not set(p.seasons['skipper']).isdisjoint(targetTRSeasons)],
                                   key=lambda x: x.tsr.ordinal(
                                       target=targetElo, alpha=200 / model.sigma),
                                   reverse=True)
            topSkipperTRSum = sum([p.tsr.ordinal(target=targetElo, alpha=200 / model.sigma)
                                   for p in topSkippersTR[:numTops]])
            topSkippersTR = [{'name': p.name, 'key': p.key,
                              'mu': p.tsr.mu, 'sigma': p.tsr.sigma} for p in topSkippersTR]

            topCrewsTR = sorted([p for p in filtered_people
                                 if p.tcr.mu != model.mu
                                 if not set(p.seasons['crew']).isdisjoint(targetTRSeasons)],
                                key=lambda x: x.tcr.ordinal(
                                    target=targetElo, alpha=200 / model.sigma),
                                reverse=True)
            topCrewTRSum = sum([p.tcr.ordinal(target=targetElo, alpha=200 / model.sigma)
                                for p in topCrewsTR[:numTops]])
            topCrewsTR = [{'name': p.name, 'key': p.key,
                           'mu': p.tcr.mu, 'sigma': p.tcr.sigma} for p in topCrewsTR]
            topRatingTR = (topSkipperTRSum + topCrewTRSum) / (numTops * 2)

            # WOMEN's Team Racing:

            topWomenSkippersTR = sorted([p for p in filtered_people
                                         if p.wtsr.mu != model.mu
                                         and p.gender == 'F'
                                         and not set(p.seasons['skipper']).isdisjoint(targetTRSeasons)
                                         ],
                                        key=lambda x: x.wtsr.ordinal(
                                            target=targetElo, alpha=200 / model.sigma),
                                        reverse=True)
            topWomenSkipperTRSum = sum([p.wtsr.ordinal(target=targetElo, alpha=200 / model.sigma)
                                        for p in topWomenSkippersTR[:numTops]])
            topWomenSkippersTR = [{'name': p.name, 'key': p.key, 'mu': p.wtsr.mu,
                                   'sigma': p.wtsr.sigma} for p in topWomenSkippersTR]

            topWomenCrewsTR = sorted([p for p in filtered_people
                                      if p.wtcr.mu != model.mu
                                      and p.gender == 'F'
                                      and not set(p.seasons['crew']).isdisjoint(targetTRSeasons)
                                      ],
                                     key=lambda x: x.wtcr.ordinal(
                                         target=targetElo, alpha=200 / model.sigma),
                                     reverse=True)
            topWomenCrewTRSum = sum([p.wtcr.ordinal(target=targetElo, alpha=200 / model.sigma)
                                     for p in topWomenCrewsTR[:numTops]])
            topWomenCrewsTR = [{'name': p.name, 'key': p.key, 'mu': p.wtcr.mu,
                                'sigma': p.wtcr.sigma} for p in topWomenCrewsTR]
            topWomenRatingTR = (topWomenSkipperTRSum +
                                topWomenCrewTRSum) / (numTops * 2)


        cursor.execute("""
                    INSERT INTO Teams (teamID, teamName, topFleetRating, topWomenRating, topTeamRating, topWomenTeamRating, avgRating, avgRatio, region, link)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,   (team, team, topRating, topWomenRating, topRatingTR, topWomenRatingTR, avg, avgRatio, region, url))            
        
        print("inserted team", team)
        
        # batch_size = 200  # adjust for your DB/network limits
        # rows_to_insert = []

        # for member in members:
        #     if member['key'] is None:
        #         print("No key for", member['name'])
        #         continue

        #     for position in ['skipper', 'crew']:
        #         for season in set(member['seasons'][position]):
        #             rows_to_insert.append((
        #                 member['key'].replace("/", "-"),
        #                 team,
        #                 season,
        #                 position,
        #                 member['raceCounts'][position][season]
        #             ))

        # # Insert in batches
        # for start in range(0, len(rows_to_insert), batch_size):
        #     batch = rows_to_insert[start:start + batch_size]
        #     try:
        #         cursor.executemany("""
        #             INSERT IGNORE INTO SailorTeams(sailorID, teamID, season, position, raceCount)
        #             VALUES(%s,%s,%s,%s,%s)
        #         """, batch)
        #         connection.commit()
        #     except mysql.connector.errors.IntegrityError as e:
        #         print("Batch insert failed:", e)
        #         raise e

        connection.commit()
        
    #     if i % 20 == 0:  # commit every 20 documents
    #         batch.commit()

    # batch.commit()
    # doc = db.collection('vars').document('eloTeams').set({"teams": teams})

    # doc = db.collection('vars').document('predTeams').set({"teams": predteams})
    # newTeams = sorted(teams, key=lambda x: x['topRating'], reverse=True)
    # return newTeams

In [ ]:
import tempfile
import os
import pandas as pd

def uploadScores(df_races_full, cursor, connection):
    # Replace NaNs with None for MySQL compatibility
    df_races_full = df_races_full.where(pd.notnull(df_races_full), None)

    # Filter valid (non-team) races
    df_filtered = df_races_full[
        (df_races_full['Scoring'] != 'team') & (~pd.isna(df_races_full['Score']))
    ].copy()

    # Compute season/regatta from raceID
    df_filtered["season"] = df_filtered["raceID"].apply(lambda x: str(x).split("/")[0])
    df_filtered["regatta"] = df_filtered["raceID"].apply(lambda x: str(x).split("/")[1])

    # Resolve sailorID and partnerID
    df_filtered["sailorID"] = df_filtered.apply(
        lambda r: r["Link"] if "Link" in r and r["Link"] not in [None, "Unknown"]
        else f"{r['Sailor']}-{r['Team']}", axis=1
    )
    df_filtered["partnerID"] = df_filtered.apply(
        lambda r: (r["partner"]["key"].replace("/", "-") if isinstance(r.get("partner"), dict) and r["partner"].get("key") else None),
        axis=1
    )

    # Fill optional / new columns with defaults if not already present
    for col, default in {
        "PartnerLink": None,
        "Position": None,
        "penalty": None,
        "Scoring": None,
        "venue": None,
        "boat": None,
        "ratingType": None,
        "oldRating": None,
        "newRating": None,
    }.items():
        if col not in df_filtered.columns:
            df_filtered[col] = default

    # Reorder and rename columns to match your MySQL table definition exactly
    df_sql = df_filtered[
        [
            "season", "regatta", "raceNum", "raceDiv",
            "sailorID", "PartnerLink", "Score", "penalty",
            "Position", "date", "Scoring", "venue", "boat",
            "ratingType", "oldRating", "newRating"
        ]
    ].copy()

    # Rename columns to match MySQL names exactly
    df_sql.rename(columns={
        "raceNum": "raceNumber",
        "raceDiv": "division",
        "PartnerLink": "partnerID",
        "Score": "score",
        "Position": "position",
        "Scoring": "scoring"
    }, inplace=True)

    # Write to a temporary CSV for bulk load
    with tempfile.NamedTemporaryFile(mode="w", suffix=".csv", delete=False) as tmp:
        csv_path = tmp.name
        df_sql.to_csv(tmp, index=False, header=False)

    try:
        # Enable local infile if not already on
        cursor.execute("SET GLOBAL local_infile = 1;")

        load_query = f"""
            LOAD DATA LOCAL INFILE '{csv_path.replace("\\", "/")}'
            INTO TABLE FleetScores
            FIELDS TERMINATED BY ','
            OPTIONALLY ENCLOSED BY '"'
            LINES TERMINATED BY '\n'
            (season, regatta, raceNumber, division, sailorID, partnerID,
             score, penalty, position, date, scoring, venue, boat,
             ratingType, oldRating, newRating)
        """
        cursor.execute(load_query)
        connection.commit()
        print(f"✅ Successfully inserted {len(df_sql)} rows into FleetScores.")

    finally:
        os.remove(csv_path)


In [4]:
[race for race in people['carter-anderson'].races if race['type'] == 'fleet'][0]
people['carter-anderson'].seasons

{'skipper': [('f23', 'UC Santa Cruz'),
  ('s24', 'UC Santa Cruz'),
  ('f24', 'Northeastern'),
  ('s25', 'Northeastern'),
  ('f25', 'Northeastern')],
 'crew': [('f23', 'UC Santa Cruz'), ('s24', 'UC Santa Cruz')]}

In [3]:
def uploadScoresBySailor(df_sailors):
    for index, row in df_sailors.iterrows():
        if index % 100 == 0:
            print(f"Uploading scores for sailor {row['Sailor']} {index}/{len(df_sailors)}")
            connection.commit()
        if row['key'] is None:
            print("No key for sailor", row['Sailor'])
            continue
        sailorID = row['key'].replace("/", "-")
        races = row['Races']
        for race in races:
            if race['type'] == 'fleet':
                try:
                    cursor.execute("""
                        INSERT IGNORE INTO FleetScores (season, regatta, raceNumber, division, sailorID, partnerID, partnerName, score, predicted, ratio, penalty, position, date, scoring, venue, boat, ratingType, oldRating, newRating, regAvg)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (race['raceID'].split("/")[0], race['raceID'].split("/")[1], race['raceID'].split("/")[2][:-1], race['raceID'].split("/")[2][-1], sailorID, race['partner']['key'], race['partner']['name'], race['score'], race['predicted'], race['ratio'], None, # penalty not tracked yet
                        race['pos'], race['date'], race['scoring'], race['venue'], None, #boat not tracked here yet
                        race['ratingType'], race['oldRating'], race['newRating'], race['regAvg']))
                except mysql.connector.errors.IntegrityError as e:
                    print(e)
            elif race['type'] == 'team':
                try:
                    cursor.execute("""
                        INSERT IGNORE INTO TRScores (season, regatta, raceNumber, round, sailorID, partnerID, partnerName, opponentTeam, opponentNick, score, outcome, predicted, penalty, position, date, venue, boat, ratingType, oldRating, newRating, regAvg)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, 
                        (race['raceID'].split("/")[0], 
                         race['raceID'].split("/")[1], 
                         race['raceID'].split("/")[2], 
                         race['round'],
                         sailorID, 
                         race['partner']['key'], 
                         race['partner']['name'], 
                         race['opponentTeam'], 
                         race['opponentNick'], 
                         race['score'], 
                         race['outcome'], 
                         race['predicted'], 
                         None,  # penalty not tracked yet, 
                         race['pos'], 
                         race['date'], 
                         race['venue'], 
                         None,  # boat not tracked here yet
                         race['ratingType'],
                         race['oldRating'],
                         race['newRating'], 
                         race['regAvg']))
                except mysql.connector.errors.IntegrityError as e:
                    print(e)        
            
                    
    cursor.execute("""UPDATE HomePageStats SET numSailors = (SELECT COUNT(*) FROM Sailors),
                         numScores = (SELECT COUNT(*) FROM FleetScores) + (SELECT COUNT(*) FROM TRScores) ,
                         numTeams = (SELECT COUNT(*) FROM Teams)
                    WHERE id = 1;""")
    connection.commit()

In [ ]:
def uploadScoresBySailor(df_sailors, connection, batch_size=3000):
    cursor = connection.cursor()
    
    fleet_rows = []
    team_rows = []

    for index, row in df_sailors.iterrows():
        if index % 100 == 0:
            print(f"Processing sailor {row['Sailor']} {index}/{len(df_sailors)}")
        if row['key'] is None:
            print("No key for sailor", row['Sailor'])
            continue
        
        sailorID = row['key'].replace("/", "-")
        races = row['Races']
        
        for race in races:
            raceID_parts = race['raceID'].split("/")
            if race['type'] == 'fleet':
                fleet_rows.append([
                    raceID_parts[0],               # season
                    raceID_parts[1],               # regatta
                    raceID_parts[2][:-1],          # raceNumber
                    raceID_parts[2][-1],           # division
                    sailorID,
                    race['partner']['key'],
                    race['partner']['name'],
                    race['score'],
                    race['predicted'],
                    race['ratio'],
                    '',                             # penalty
                    race['pos'],
                    race['date'],
                    race['scoring'],
                    race['venue'],
                    '',                             # boat
                    race['ratingType'],
                    race['oldRating'],
                    race['newRating'],
                    race['regAvg']
                ])
            elif race['type'] == 'team':
                team_rows.append([
                    raceID_parts[0],               # season
                    raceID_parts[1],               # regatta
                    raceID_parts[2],               # raceNumber
                    race['round'],
                    sailorID,
                    race['partner']['key'],
                    race['partner']['name'],
                    race['opponentTeam'],
                    race['opponentNick'],
                    race['score'],
                    race['outcome'],
                    race['predicted'],
                    '',                             # penalty
                    race['pos'],
                    race['date'],
                    race['venue'],
                    '',                             # boat
                    race['ratingType'],
                    race['oldRating'],
                    race['newRating'],
                    race['regAvg']
                ])
    
    # Function to insert in batches
    def batch_insert(cursor, table_name, columns, data):
        for start in range(0, len(data), batch_size):
            print("Inserting " + start + "/" + len(data))
            batch = data[start:start + batch_size]
            placeholders = ",".join(["%s"] * len(columns))
            sql = f"INSERT IGNORE INTO {table_name} ({','.join(columns)}) VALUES ({placeholders})"
            cursor.executemany(sql, batch)
            connection.commit()
    
    fleet_columns = [
        'season', 'regatta', 'raceNumber', 'division', 'sailorID', 'partnerID', 'partnerName',
        'score', 'predicted', 'ratio', 'penalty', 'position', 'date', 'scoring', 'venue',
        'boat', 'ratingType', 'oldRating', 'newRating', 'regAvg'
    ]
    team_columns = [
        'season', 'regatta', 'raceNumber', 'round', 'sailorID', 'partnerID', 'partnerName',
        'opponentTeam', 'opponentNick', 'score', 'outcome', 'predicted', 'penalty', 'position',
        'date', 'venue', 'boat', 'ratingType', 'oldRating', 'newRating', 'regAvg'
    ]
    
    print("Inserting FleetScores...")
    batch_insert(cursor, "FleetScores", fleet_columns, fleet_rows)
    
    print("Inserting TRScores...")
    batch_insert(cursor, "TRScores", team_columns, team_rows)
    
    # Update homepage stats
    cursor.execute("""
        UPDATE HomePageStats SET 
            numSailors = (SELECT COUNT(*) FROM Sailors),
            numScores = (SELECT COUNT(*) FROM FleetScores) + (SELECT COUNT(*) FROM TRScores),
            numTeams = (SELECT COUNT(*) FROM Teams)
        WHERE id = 1;
    """)
    
    connection.commit()
    cursor.close()
    print("Upload complete.")


In [32]:
import importlib
import fullfile
importlib.reload(fullfile)
from fullfile import uploadSailors 

if doUpload:
    uploadSailors(people, cursor, connection)
    # connection.commit() 
    # teams = uploadTeams(df_sailors, df_races_full, people, cursor, connection)
    # uploadScores(df_races_full, cursor, connection)
    # uploadScoresBySailor(df_sailors, connection)
    # uploadTops(people)
    # uploadAllSailors(people)
    # connection.commit() 

16641
Uploading sailors 0 to 299... 785
Uploading sailors 300 to 599... 719
Uploading sailors 600 to 899... 696
Uploading sailors 900 to 1199... 701
Uploading sailors 1200 to 1499... 720
Uploading sailors 1500 to 1799... 682
Uploading sailors 1800 to 2099... 752
Uploading sailors 2100 to 2399... 813
Uploading sailors 2400 to 2699... 885
Uploading sailors 2700 to 2999... 910
Uploading sailors 3000 to 3299... 948
Uploading sailors 3300 to 3599... 936
Uploading sailors 3600 to 3899... 1010
Uploading sailors 3900 to 4199... 1002
Uploading sailors 4200 to 4499... 964
Uploading sailors 4500 to 4799... 1059
Uploading sailors 4800 to 5099... 1071
Uploading sailors 5100 to 5399... 972
Uploading sailors 5400 to 5699... 1014
Uploading sailors 5700 to 5999... 984
Uploading sailors 6000 to 6299... 1054
Uploading sailors 6300 to 6599... 1008
Uploading sailors 6600 to 6899... 1005
Uploading sailors 6900 to 7199... 1047
Uploading sailors 7200 to 7499... 984
Uploading sailors 7500 to 7799... 1018
Uploa

In [28]:
cursor.close()
connection.close()

In [ ]:
[{'score': 11, 'pos': 'Skipper', 'predicted': 15, 'ratio': 0.4117647058823529, 'change': 34.99197330037646, 'regAvg': 1337.4778010123323, 'cross': 0, 'outLinks': 0, 'sr': 1034.9919733003765, 'srmu': 26.288013646593356, 'srsig': 8.276671586359223, 'ratingType': 'sr', 'oldRating': 1000.0, 'newRating': 1034.9919733003765, 'cr': 1000.0, 'wsr': 1000.0, 'wcr': 1000.0, 'tsr': 1000.0, 'tcr': 1000.0, 'wtsr': 1000.0, 'wtcr': 1000.0, 'womens': False, 'date': Timestamp('2021-09-11 00:00:00'), 'partner': {'name': 'Maaya Prasad', 'key': 'maaya-prasad'}, 'venue': 'Yale', 'raceID': 'f21/2021-harry-anderson-jr/1B', 'type': 'fleet', 'scoring': '2 Divisions'}]
 

In [43]:
temp = [row['raceID'] for index, row in df_races_full.iterrows() if row['Scoring'] != 'team' and row['raceID'].split("/")[2][-1] not in ("A", "B", "C")]
temp

[]

In [ ]:
df_races_full
df_sailors

In [34]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore


cred = credentials.Certificate("thecrowsnestapp-creds.json")
# firebase_admin.initialize_app(cred)
db = firestore.client()

users_ref = db.collection("users")

# Get all documents
docs = users_ref.stream()

# Print or store them
users = []
for doc in docs:
    data = doc.to_dict()
    data["id"] = doc.id  # include document ID
    users.append(data)

print(users)

[{'techscoreLink': 'https://scores.collegesailing.org/sailors/ray-mcdonald', 'photoURL': 'https://lh3.googleusercontent.com/a/ACg8ocJNWyaL2aLaKn8dpjdLZD3WpEgjtqY7GE6Z0haW6269IMZ_mP1p=s96-c', 'displayName': 'Ray', 'teams': [], 'username': 'sakana', 'techscoreID': '3137779', 'id': '3yFqUXDhLSQtzCsRrHn8OI8FQxo1'}, {'following': [{'targetName': 'Carter Anderson', 'targetKey': 'carter-anderson'}, {'targetName': 'Elliott Chalcraft', 'targetKey': 'elliott-chalcraft'}, {'targetName': 'Sabrina Anderson', 'targetKey': 'sabrina-anderson'}], 'photoURL': 'https://lh3.googleusercontent.com/a/AEdFTp7DfXgwpbOq8_qVhRY3PYDBoD1_nUqs0dkpWu4RAxI=s96-c', 'displayName': 'Carter', 'teams': ['bainbridge', 'ucsc'], 'pro': True, 'techscoreID': '3135057', 'techscoreLink': 'https://scores.collegesailing.org/sailors/carter-anderson/', 'username': '7fi', 'id': '4y9mckQ7d9Zn2OvNsPjfRFSv1ol1'}, {'photoURL': 'https://lh3.googleusercontent.com/a/ACg8ocLhIAoGOIYIALEjWnLccZRh_v-NfgNKFwW8mJZ9k-uJERq4HCLT=s96-c', 'displayNa

In [43]:
for i, user in enumerate(users):
    # for key, value in user.items():
    # print(user.keys())
    cursor.execute("""
        INSERT INTO Users (userID, username, displayName, techscoreLink, techscoreID, photoURL, pro)
        VALUES (%s, %s, %s, %s, %s, %s, %s)""", (user['id'], user['username'], user['displayName'], user['techscoreLink'] if 'techscoreLink' in user.keys() else '', user['techscoreID'] if 'techscoreID' in user.keys() else 0, user['photoURL'], user['pro'] if 'pro' in user.keys() else False))
    # if 'following' in user:
    #     print(user['id'], user['following'])
    #     for follow in user['following']:
    #         cursor.execute("""
    #             INSERT INTO SailorFollows (sailorID, sailorName, userID)
    #             VALUES (%s, %s, %s)
    #         """, (follow['targetKey'], follow['targetName'], user['id']))
        
connection.commit()

In [24]:
rivals = df_sailors[df_sailors['key'] == 'carter-anderson'].iloc[0]['Rivals']
sailorID = 'carter-anderson'
print(rivals['Skipper'])
for position in ['Skipper', 'Crew']:
    for key, values in rivals[position].items():
        print(key, values)
        for season in values['races'].keys():
            
            cursor.execute('''INSERT INTO SailorRivals (sailorID, rivalID, rivalName,  rivalTeam, position, season, raceCount, winCount) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)''', (sailorID, key, values['name'], values['team'], position, season, values['races'][season], values['wins'][season]))
            
connection.commit()

{'eric-anderson-2026': {'name': 'Eric Anderson', 'races': {'f23': 6, 's24': 3}, 'team': 'Hawaii', 'wins': {'f23': 1, 's24': 0}}, 'henry-boeger': {'name': 'Henry Boeger', 'races': {'f23': 6, 's24': 11}, 'team': 'UC Santa Barbara', 'wins': {'f23': 2, 's24': 1}}, 'nicholas-mueller': {'name': 'Nicholas Mueller', 'races': {'f23': 6, 's24': 20}, 'team': 'Cal Maritime', 'wins': {'f23': 3, 's24': 0}}, 'hudson-mayfield': {'name': 'Hudson Mayfield', 'races': {'f23': 6, 's24': 12}, 'team': 'Southern Cal', 'wins': {'f23': 4, 's24': 2}}, 'braedon-hansen': {'name': 'Braedon Hansen', 'races': {'f23': 6, 's24': 8}, 'team': 'UC Davis', 'wins': {'f23': 5, 's24': 6}}, 'max-case': {'name': 'Max Case', 'races': {'f23': 6, 's24': 12}, 'team': 'Cal Poly', 'wins': {'f23': 4, 's24': 7}}, 'maxwell-miller-2025': {'name': 'Maxwell Miller', 'races': {'f23': 6, 's24': 10}, 'team': 'Washington', 'wins': {'f23': 3, 's24': 1}}, 'nicholas-chesemore': {'name': 'Nicholas Chesemore', 'races': {'f23': 6}, 'team': 'Northwes